In [1]:
import os
import re
import string
import pandas as pd
import cleantext
from pandarallel import pandarallel
from tqdm import tqdm
import langdetect

In [2]:
DATA_PATH = "../data"

CHAR_MAP = {
    "â": "a",
    "İ": "i",
    "I": "ı",
    "Â": "a",
    "Î": "i",
    "î": "i",
    "ì": "i",
    "í": "i",
    "û": "u",
    "Û": "u", 
    "|": "",
    "¦": "",
    "~": "",
    "°": "",
    "^": "",
    "+": "",
    "<": "",
    "=": "",
    ">": "",
    "±": "",
    "ĝ": "ğ",
    "ܧ": "",
    "'ާ": "",
    "ޱ": "",
    "绫": "",
    "ȥ": "",
    "ǥ": "",
    ". com. tr": ".com.tr",
    ". com": ".com",
    " km.": "km",
    "@hotmail. com": "@hotmail.com",
    " hotmail. com": "@hotmail.com"
}

EMAIL_PATTERNS = [
    "gmail",
    "hotmail",
    "yahoo",
    ".com",
]

In [3]:
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
class Cleaner():
    def __init__(self):
        pass
    
    def __clean_text(self, text, clean_punc):
        for char, replaced_char in CHAR_MAP.items():
            text = text.replace(char, replaced_char) 
        text = cleantext.clean(
            text,
            to_ascii=False,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=True,
            no_numbers=True,
            no_digits=True,
            no_currency_symbols=True,
            no_punct=clean_punc,
            no_emoji=True,
            replace_with_url=' ',
            replace_with_email=' ',
            replace_with_phone_number=' ',
            replace_with_punct='',
            replace_with_number='',
            replace_with_digit='',
            replace_with_currency_symbol=''
        )
        text = " ".join([token for token in text.split() if len(token) > 1])
        if not clean_punc:
            for email_token in EMAIL_PATTERNS:
                text = " ".join([token for token in text.split() if email_token not in token])
        return text
    
    def __remove_non_turkish_contents(self, df):
        df = df[df["content"] != ""]
        df["suggested_lang"] = df["content"].parallel_apply(lambda content: langdetect.detect(content))
        df = df[df["suggested_lang"] == "tr"]
        df = df.drop("suggested_lang", axis=1)
        return df
    
    def __split_sentences(self, text):
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
        return sentences
    
    def __remove_punctuations(self, sentences):
        sentences = [self.__clean_text(sent, clean_punc=True) for sent in sentences]
        return sentences
    
    def __clean_bad_sentences(self, sentences):
        sentences = [sent for sent in sentences if len(sent.split()) > 1 and len(sent) >= 5]
        return sentences
    
    def process_df(self, df):
        for col in ["pub_name", "title", "content"]:
            clean_punc = False if col == "content" else True
            df[col] = df[col].parallel_apply(lambda text: self.__clean_text(text, clean_punc))
        df = self.__remove_non_turkish_contents(df)
        df["sentences"] = df["content"].parallel_apply(lambda text: self.__split_sentences(text))
        df["sentences"] = df["sentences"].parallel_apply(lambda sentences: self.__remove_punctuations(sentences))
        df["sentences"] = df["sentences"].parallel_apply(lambda sentences: self.__clean_bad_sentences(sentences))
        return df

In [5]:
text_cleaner = Cleaner()

In [6]:
df = pd.read_csv(os.path.join(DATA_PATH, 'data_all_2020-03-19.csv'), sep='|', encoding='utf-8')
#df = df[df["title"] != "TRUMP IRKÇILIKTAN VAZGEÇMİYOR"]

In [7]:
df_cleaned = text_cleaner.process_df(df.copy())

/home/tabilab/anaconda3/envs/hate_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
for i, row in tqdm(df_cleaned.iterrows()):
    if langdetect.detect(row["content"]) != "tr":
        display(row)

25066it [03:36, 115.93it/s]


In [12]:
df_cleaned

,date,pub_name,type,title,content,Label,sentences
0,05 Ekim 2015 Pazartesi,akşam,ulusal,sahte polislerin kuryesi yakalandı,sahte polislerin kuryesi yakalandı antalya'da ...,hate,[sahte polislerin kuryesi yakalandı antalyada ...
1,11 Eylül 2015 Cuma,akşam,ulusal,kürt üz ama hain değiliz,kürt'üz ama hain değiliz suriye sınırında devr...,hate,[kürtüz ama hain değiliz suriye sınırında devr...
2,25 Eylül 2015 Cuma,akşam,ulusal,suriyeli gelinden altın vurgunu,kuyumcuda altın alırken fotoğraf çektirdi. sur...,hate,"[kuyumcuda altın alırken fotoğraf çektirdi, su..."
3,07 Eylül 2015 Pazartesi,anayurt,ulusal,mustafa nevruz sınacı,"mustafa nevruz sınacı lgercek. abd'li yahudi, ...",hate,"[mustafa nevruz sınacı lgercek, abdli yahudi b..."
4,21 Eylül 2015 Pazartesi,anayurt,ulusal,mustafa nevruz sınacı,"mustafa nevruz sınacı yazıyor, gercek. abd'li ...",hate,"[mustafa nevruz sınacı yazıyor gercek, abdli y..."
...,...,...,...,...,...,...,...
25071,02 Mayıs 2014 Cuma,yeni asya,hepsi,amnesty ınternational ve global ahlaksızlık,doğu veya batı s. bulut@saidnursi. de amnesty ...,hate,"[doğu veya batı, de amnesty ınternational ve g..."
25072,26 Mart 2014 Çarşamba,yeni konya,hepsi,çanakkale asla unutulmamalı llnutturulmamalı,"çanakkale, asla unutulmamalı, llnutturulmamalı...",hate,[çanakkale asla unutulmamalı llnutturulmamalı ...
25073,04 Nisan 2014 Cuma,yeni mesaj,hepsi,sömürü projesi olarak bop,btp genel başkanı prof. dr. haydar bas ın kale...,hate,"[btp genel başkanı prof, haydar bas ın kalemin..."
25074,24 Şubat 2014 Pazartesi,yeni mesaj,hepsi,doğruluş zeminimiz helali bir millet istiklali...,prof. dr. nurullah çetin doğruluş zeminimiz: '...,hate,[nurullah çetin doğruluş zeminimiz helali bir ...


In [18]:
#df_cleaned[df_cleaned["content"].str.contains("mail")]#.sample(10)

In [17]:
sample_id = 101
for sent in df_cleaned.loc[sample_id, "sentences"]:
    print(sent)
    print("--------------")
print(df_cleaned.loc[sample_id, "content"])
print(df_cleaned.loc[sample_id, "Label"])

ingiliz zoru görünce çark etti ıgh ttlırmttlatl om ıunmy the jıtevıılg txcorlıvigtı pollruk uants toquıt europe shock sıırvev frırr
--------------
jest showseuno itisyss anurnııissam lıihıılefora mileslone yapılan son anketlere göre ingiliz halkının yüzde abden çıkmak istiyor
--------------
son dönemde avrupa birliği ülkelerine yaşanan mülteci akını ingiliz halkını tedirgin etti
--------------
ingiliz daily mail gazetesi icin yapılan bir anket ingiltere halkının çoğunun avrupa birliği ab üyeliğinden artık çıkılmasını desteklediğini ortaya koydu
--------------
son ankete katılanların yüzde ingilterenin abden çıkmasını desteklerken yüzde ülkelerinin birlik içinde kalmasından yana olduğunu belirtti
--------------
yüzde si ise henüz net bir karar ünd syrians wanted ai iüic sjsıo ıc
--------------
wx wmr frnr sahibi değil
--------------
bir önceki ankette ise ingiltere halkının coğu ülkeleri nin ab içinde kalmasından yana olduklarını belirtmişti
--------------
ingilterenin abden çıkması ya 

In [8]:
alphabet = set()
for text in df_cleaned["content"].values:
    alphabet = set.union(alphabet, set(text))

In [9]:
alphabet

{' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ç',
 'ö',
 'ü',
 'ĝ',
 'ğ',
 'ı',
 'ş',
 'ܧ',
 'ާ'}

In [20]:
alphabet = set()
for sentences in tqdm(df_cleaned["sentences"].values):
    for sent in sentences:
        alphabet = set.union(alphabet, set(sent))

100%|██████████| 25066/25066 [00:02<00:00, 9948.36it/s] 


In [19]:
for sentences in tqdm(df_cleaned["sentences"].values):
    for sent in sentences:
        if 'ȥ' in sent:
            print(sent)

100%|██████████| 25066/25066 [00:00<00:00, 394645.78it/s]

alman auȥubath


In [21]:
alphabet

{' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ç',
 'ö',
 'ü',
 'ğ',
 'ı',
 'ş',
 'ȥ',
 'ާ'}

In [22]:
print(df_cleaned.shape)
df_cleaned = df_cleaned[df_cleaned["content"].notnull()]
print(df_cleaned.shape)
df_cleaned.to_csv('../data/data_cleaned_sentences_2020-04-10.csv', sep='|', index=False)

(25066, 7)
(25066, 7)


In [24]:
df_cleaned["sentences"] = df_cleaned["sentences"].parallel_apply(lambda sentences: "<SEP>".join(sentences))

In [27]:
df_cleaned = df_cleaned[df_cleaned["Label"] == "hate"]
df_cleaned

,date,pub_name,type,title,content,Label,sentences
0,05 Ekim 2015 Pazartesi,akşam,ulusal,sahte polislerin kuryesi yakalandı,sahte polislerin kuryesi yakalandı antalya'da ...,hate,sahte polislerin kuryesi yakalandı antalyada k...
1,11 Eylül 2015 Cuma,akşam,ulusal,kürt üz ama hain değiliz,kürt'üz ama hain değiliz suriye sınırında devr...,hate,kürtüz ama hain değiliz suriye sınırında devri...
2,25 Eylül 2015 Cuma,akşam,ulusal,suriyeli gelinden altın vurgunu,kuyumcuda altın alırken fotoğraf çektirdi. sur...,hate,kuyumcuda altın alırken fotoğraf çektirdi<SEP>...
3,07 Eylül 2015 Pazartesi,anayurt,ulusal,mustafa nevruz sınacı,"mustafa nevruz sınacı lgercek. abd'li yahudi, ...",hate,mustafa nevruz sınacı lgercek<SEP>abdli yahudi...
4,21 Eylül 2015 Pazartesi,anayurt,ulusal,mustafa nevruz sınacı,"mustafa nevruz sınacı yazıyor, gercek. abd'li ...",hate,mustafa nevruz sınacı yazıyor gercek<SEP>abdli...
...,...,...,...,...,...,...,...
25071,02 Mayıs 2014 Cuma,yeni asya,hepsi,amnesty ınternational ve global ahlaksızlık,doğu veya batı s. bulut@saidnursi. de amnesty ...,hate,doğu veya batı<SEP>de amnesty ınternational ve...
25072,26 Mart 2014 Çarşamba,yeni konya,hepsi,çanakkale asla unutulmamalı llnutturulmamalı,"çanakkale, asla unutulmamalı, llnutturulmamalı...",hate,çanakkale asla unutulmamalı llnutturulmamalı k...
25073,04 Nisan 2014 Cuma,yeni mesaj,hepsi,sömürü projesi olarak bop,btp genel başkanı prof. dr. haydar bas ın kale...,hate,btp genel başkanı prof<SEP>haydar bas ın kalem...
25074,24 Şubat 2014 Pazartesi,yeni mesaj,hepsi,doğruluş zeminimiz helali bir millet istiklali...,prof. dr. nurullah çetin doğruluş zeminimiz: '...,hate,nurullah çetin doğruluş zeminimiz helali bir m...


In [29]:
df_cleaned.to_excel('../data/data_cleaned_SEP_sentences_2020-04-10_only_hate.xlsx', index=False)